In [2]:
import pandas as pd
import numpy as np

In [3]:
course_information = pd.read_csv('course_information.csv')
course_posts = pd.read_csv('course_posts.csv')
course_threads = pd.read_csv('course_threads.csv')

# Feature 3, 4

* Maximum breadth
* Index Max Breadth (Imaxb)  (post_id column name)

In [4]:
course_posts['message_tuple'] = course_posts[['post_id', 'parent_id', 'user_id'
                                             ]].apply(tuple, axis=1)
course_posts.head()

,post_id,thread_id,course_id,parent_id,order,user_id,user_type,post_time,relative_t,votes,num_words,forum_id,message_tuple
0,3,2,analysenumerique-001,0,1,0,Anonymous,1358942448,-0.404753,2,23,13,"(3, 0, 0)"
1,4,2,analysenumerique-001,0,2,3992067770,Instructor,1359109877,-0.373994,0,15,13,"(4, 0, 3992067770)"
2,7,2,analysenumerique-001,0,3,3992067770,Instructor,1359129778,-0.370338,1,9,13,"(7, 0, 3992067770)"
3,8,2,analysenumerique-001,0,4,0,Anonymous,1359130161,-0.370267,0,2,13,"(8, 0, 0)"
4,15,2,analysenumerique-001,0,5,3992067770,Instructor,1359214077,-0.354851,0,31,13,"(15, 0, 3992067770)"


In [5]:
cp_new = course_posts.groupby(['course_id', 'thread_id']
                    )['message_tuple'].apply(list).reset_index(
    name='group_tuple')
cp_new.head()

,course_id,thread_id,group_tuple
0,analysenumerique-001,2,"[(3, 0, 0), (4, 0, 3992067770), (7, 0, 3992067..."
1,analysenumerique-001,3,"[(5, 0, 0), (6, 0, 3992067770), (9, 0, 9149558..."
2,analysenumerique-001,4,"[(11, 0, 7849522082), (32, 0, 2403613940)]"
3,analysenumerique-001,7,"[(28, 0, 3992067770), (37, 0, 2948794923), (38..."
4,analysenumerique-001,8,"[(29, 0, 6939953186), (33, 0, 85995101), (35, ..."


In [6]:
from typing import List, Tuple
def get_thread_len(posts: List[Tuple[int, int, int]]):
    '''
    each post is a tuple of ('post_id', 'parent_id', 'user_id')
    
    returns max_depth, post_index
    '''
    leaf_threads = [[p] for p in posts if p[1] != 0]
    lookup_nodes = {p[0]: p for p in posts}  
    if len(leaf_threads) == 0:
        return 0, 0
    
    for leaf_thread in leaf_threads:
        safety_counter = 0
        while leaf_thread[-1][1] != 0:
            head = leaf_thread[-1]
            safety_counter += 1
            key = head[1]
            if key in lookup_nodes:
                val = lookup_nodes[key]
                leaf_thread.append(val)
            if safety_counter > 20:
#                 print('stuck in infinite loop')
                break
 
    post_lengths = {len(t):t for t in leaf_threads}
    max_candidate_list = [k for (k, v) in post_lengths.items() if k < 20]
    if len(max_candidate_list) == 0:
        return 0, 0
    max_post_len = max(max_candidate_list)
    max_item = post_lengths[max_post_len]
    return max_post_len, max_item[0][0]

# print(get_thread_len([(5, 0, 0), (6, 0, 3992067770), (9, 0, 9149558)]))
# print(get_thread_len([(1, 0, 1), (2, 0, 2), (3, 2, 1)]))

test = [(49, 0, 7447716722), (52, 0, 7447716722), (53, 0, 8574831658), (56, 0, 4601308412), (63, 0, 1994296489), (66, 0, 8066812249), 
        (71, 0, 4374260282), (73, 0, 3097917812), (75, 0, 5914379209), (76, 0, 5914379209), (77, 0, 1183881388), (79, 0, 9073042454), 
        (80, 0, 4705996654), (81, 0, 2748090138), (83, 0, 9234683582), (87, 0, 6248202036), (94, 0, 2724430131), (97, 0, 6335887787), 
        (101, 0, 5854779539), (103, 0, 5914715154), (107, 0, 2235733715), (110, 0, 5240302809), (27, 77, 9528639982), (115, 0, 9528639982), 
        (28, 115, 9528639982), (116, 0, 365877667), (29, 73, 8066812249), (30, 52, 6777831602), (118, 0, 731900228), (123, 0, 5340944374), 
        (124, 0, 5130684722), (33, 115, 7447716722), (127, 0, 1992689522), (128, 0, 6398729272), (130, 0, 5810744167), (143, 0, 4199112615), 
        (50, 143, 8066812249), (147, 0, 6552212376), (149, 0, 200534506), (156, 0, 9750201619), (64, 77, 7053515305), (158, 0, 7053515305), 
        (159, 0, 1408339376), (160, 0, 7672336125), (70, 87, 6524613967), (73, 71, 8784882853), (173, 0, 2155207307), (174, 0, 4174537809), 
        (75, 143, 4374260282), (76, 143, 8066812249), (77, 143, 4374260282), (177, 0, 3377158954), (178, 0, 1865615938), (180, 0, 6672296405), 
        (79, 71, 4374260282), (80, 143, 7447716722), (181, 0, 2972886040), (81, 143, 4374260282), (186, 0, 913988747), (208, 0, 6489280137), 
        (213, 0, 5839942379), (109, 147, 9528639982), (215, 0, 2689268877), (110, 147, 6030314646), (218, 0, 6030314646), (131, 178, 4374260282), 
        (131, 178, 4374260282), (132, 178, 0), (132, 178, 0), (139, 80, 183735521), (141, 81, 183735521), (269, 0, 183735521), (142, 81, 7447716722), 
        (143, 81, 183735521), (150, 71, 8784882853), (153, 143, 7959197517), (280, 0, 9438585628), (159, 143, 7447716722), (162, 71, 4374260282), 
        (163, 143, 4374260282), (164, 130, 7959197517), (294, 0, 7480622898), (165, 178, 913988747), (165, 178, 913988747), (308, 0, 3353891574), 
        (317, 0, 1397098893), (328, 0, 1103875643), (395, 0, 6107921315), (224, 395, 7053515305), (225, 395, 183735521), (226, 395, 6489280137), 
        (227, 395, 183735521), (228, 395, 6107921315), (400, 0, 7053515305), (229, 395, 7053515305), (234, 395, 6489280137), (236, 395, 4028146443), 
        (237, 395, 2507991908), (239, 395, 4028146443), (243, 395, 7959197517), (244, 395, 7053515305), (417, 0, 8096486767), (362, 97, 2732979732), 
        (363, 123, 2732979732), (364, 395, 2732979732), (574, 0, 2732979732), (366, 574, 183735521), (368, 574, 2732979732), (590, 0, 7934673810), 
        (611, 0, 6284451913), (613, 0, 6284451913), (624, 0, 6654646229), (630, 0, 5908668178), (730, 0, 8538074607), (760, 0, 1630848973), (784, 0, 5953258055), (548, 760, 4374260282)]
print(get_thread_len(test))

(3, 29)


In [7]:
# course_posts[(course_posts['post_id'] == 81 ) & \
#             (course_posts['parent_id'] == 143) & \
#              (course_posts['user_id'] == 4374260282)
# ]
df = course_posts[(course_posts['thread_id'] == 17 ) & \
            (course_posts['course_id'] == 'automata-002')
]
df.head(20)

,post_id,thread_id,course_id,parent_id,order,user_id,user_type,post_time,relative_t,votes,num_words,forum_id,message_tuple
3999,49,17,automata-002,0,1,7447716722,Student,1383589009,0.018135,1,21,2,"(49, 0, 7447716722)"
4000,52,17,automata-002,0,2,7447716722,Student,1383589677,0.018319,4,175,2,"(52, 0, 7447716722)"
4001,53,17,automata-002,0,3,8574831658,Student,1383590147,0.018449,0,53,2,"(53, 0, 8574831658)"
4002,56,17,automata-002,0,4,4601308412,Student,1383590436,0.018528,0,51,2,"(56, 0, 4601308412)"
4003,63,17,automata-002,0,5,1994296489,Student,1383594089,0.019535,4,9,2,"(63, 0, 1994296489)"
4004,66,17,automata-002,0,6,8066812249,Student,1383594612,0.019679,1,30,2,"(66, 0, 8066812249)"
4005,71,17,automata-002,0,7,4374260282,Student,1383598317,0.020700,3,51,2,"(71, 0, 4374260282)"
4006,73,17,automata-002,0,8,3097917812,Student,1383599214,0.020947,3,22,2,"(73, 0, 3097917812)"
4007,75,17,automata-002,0,9,5914379209,Student,1383600041,0.021175,0,72,2,"(75, 0, 5914379209)"
4008,76,17,automata-002,0,10,5914379209,Student,1383601791,0.021658,0,0,2,"(76, 0, 5914379209)"


In [8]:
cp_new['thread_len_tuple'] = cp_new['group_tuple'].apply(get_thread_len)
cp_new['thread_len'] = cp_new['thread_len_tuple'].apply(lambda x: x[1])
cp_new['max_len_post'] = cp_new['thread_len_tuple'].apply(lambda x: x[0])
cp_new.head()

,course_id,thread_id,group_tuple,thread_len_tuple,thread_len,max_len_post
0,analysenumerique-001,2,"[(3, 0, 0), (4, 0, 3992067770), (7, 0, 3992067...","(3, 4)",4,3
1,analysenumerique-001,3,"[(5, 0, 0), (6, 0, 3992067770), (9, 0, 9149558...","(2, 21)",21,2
2,analysenumerique-001,4,"[(11, 0, 7849522082), (32, 0, 2403613940)]","(0, 0)",0,0
3,analysenumerique-001,7,"[(28, 0, 3992067770), (37, 0, 2948794923), (38...","(0, 0)",0,0
4,analysenumerique-001,8,"[(29, 0, 6939953186), (33, 0, 85995101), (35, ...","(2, 6)",6,2


# Feature 1, 2

* Number of posts (np): number of posts in a thread.
* Number of comments (nc): number of Comments in a thread

In [9]:
course_posts['is_post'] = course_posts.parent_id==0

#using groupby.count function to group the variables 

tmp = course_posts.groupby(['course_id', 'thread_id', 'is_post','forum_id']).count()[['post_id']].reset_index()

# renaming the varibale 'post_id as count

tmp['count'] = tmp['post_id']
tmp.drop('post_id', axis=1, inplace=True)
tmp.head()

,course_id,thread_id,is_post,forum_id,count
0,analysenumerique-001,2,False,13,2
1,analysenumerique-001,2,True,13,9
2,analysenumerique-001,3,False,13,3
3,analysenumerique-001,3,True,13,16
4,analysenumerique-001,4,True,13,2


In [10]:
#generating a new column with the name num_posts and assigning it value as 0

tmp['num_posts'] = 0

#checking if it's a post if yes replacing it with the value from count

tmp.loc[tmp.is_post == True, 'num_posts'] = tmp['count']
tmp.head()

,course_id,thread_id,is_post,forum_id,count,num_posts
0,analysenumerique-001,2,False,13,2,0
1,analysenumerique-001,2,True,13,9,9
2,analysenumerique-001,3,False,13,3,0
3,analysenumerique-001,3,True,13,16,16
4,analysenumerique-001,4,True,13,2,2


In [11]:
#if the condition is false inserting values form count to num_comments

tmp['num_comments'] = 0

tmp.loc[tmp.is_post==False, 'num_comments'] = tmp['count']
tmp.head()

,course_id,thread_id,is_post,forum_id,count,num_posts,num_comments
0,analysenumerique-001,2,False,13,2,0,2
1,analysenumerique-001,2,True,13,9,9,0
2,analysenumerique-001,3,False,13,3,0,3
3,analysenumerique-001,3,True,13,16,16,0
4,analysenumerique-001,4,True,13,2,2,0


In [12]:

#using groupby.max and resetting the index

tmp = tmp.groupby(['course_id', 'thread_id']).max().reset_index()

tmp.head()

,course_id,thread_id,is_post,forum_id,count,num_posts,num_comments
0,analysenumerique-001,2,True,13,9,9,2
1,analysenumerique-001,3,True,13,16,16,3
2,analysenumerique-001,4,True,13,2,2,0
3,analysenumerique-001,7,True,13,3,3,0
4,analysenumerique-001,8,True,13,8,8,1


## Merging both the dataframes

In [13]:
merged = cp_new.merge(tmp, on=['course_id', 'thread_id'])

In [14]:
merged[:5]

,course_id,thread_id,group_tuple,thread_len_tuple,thread_len,max_len_post,is_post,forum_id,count,num_posts,num_comments
0,analysenumerique-001,2,"[(3, 0, 0), (4, 0, 3992067770), (7, 0, 3992067...","(3, 4)",4,3,True,13,9,9,2
1,analysenumerique-001,3,"[(5, 0, 0), (6, 0, 3992067770), (9, 0, 9149558...","(2, 21)",21,2,True,13,16,16,3
2,analysenumerique-001,4,"[(11, 0, 7849522082), (32, 0, 2403613940)]","(0, 0)",0,0,True,13,2,2,0
3,analysenumerique-001,7,"[(28, 0, 3992067770), (37, 0, 2948794923), (38...","(0, 0)",0,0,True,13,3,3,0
4,analysenumerique-001,8,"[(29, 0, 6939953186), (33, 0, 85995101), (35, ...","(2, 6)",6,2,True,13,8,8,1


# Final output dataframe with all the thread features

In [15]:
merged = merged.drop('forum_id',axis=1)
merged = merged.drop('group_tuple',axis=1)
merged = merged.drop('is_post',axis=1)
merged = merged.drop('count',axis=1)
merged = merged.drop('thread_len_tuple',axis=1)
merged[:5]

,course_id,thread_id,thread_len,max_len_post,num_posts,num_comments
0,analysenumerique-001,2,4,3,9,2
1,analysenumerique-001,3,21,2,16,3
2,analysenumerique-001,4,0,0,2,0
3,analysenumerique-001,7,0,0,3,0
4,analysenumerique-001,8,6,2,8,1
